<a href="https://colab.research.google.com/github/aravindh698/rep2/blob/master/analytics%20vidhya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [252]:
import pandas as pd
import numpy as np

In [253]:
df1=pd.read_csv('/content/train_v9rqX0R.csv',index_col='Item_Identifier')
df2=pd.read_csv('/content/test_AbJTz2l.csv',index_col='Item_Identifier')
df1.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
Item_Identifier,,,,,,,,,,,
FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [254]:
df1.isnull().sum()

Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [255]:
df1.Item_Weight.value_counts()
df1.Item_Weight.fillna(12.150,inplace=True)

In [256]:
df1.Outlet_Size.value_counts()
df1.Outlet_Size.fillna('Medium',inplace=True)

In [257]:
df2.isnull().sum()

Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [258]:
df2.Item_Weight.value_counts()
df2.Item_Weight.fillna(10.50,inplace=True)

In [259]:
df2.Outlet_Size.value_counts()
df2.Outlet_Size.fillna('Medium',inplace=True)

In [260]:
df1.dtypes

Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [261]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()


In [262]:
le.fit_transform(df1['Item_Fat_Content'])
df1.Item_Fat_Content.value_counts()
df1['Item_Fat_Content']=df1['Item_Fat_Content'].map({'Low Fat':1,'Regular':2,'LF':1,'reg':2,'low fat':1})

In [263]:
df1['Outlet_Size']=le.fit_transform(df1['Outlet_Size'])
df1['Outlet_Location_Type']=le.fit_transform(df1['Outlet_Location_Type'])
df1['Outlet_Type']=le.fit_transform(df1['Outlet_Type'])
df1['Item_Type']=le.fit_transform(df1['Item_Type'])
df1['Outlet_Identifier']=le.fit_transform(df1['Outlet_Identifier'])

In [264]:
df1.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
Item_Identifier,,,,,,,,,,,
FDA15,9.30,1,0.016047,4,249.8092,9,1999,1,0,1,3735.1380
DRC01,5.92,2,0.019278,14,48.2692,3,2009,1,2,2,443.4228
FDN15,17.50,1,0.016760,10,141.6180,9,1999,1,0,1,2097.2700
FDX07,19.20,2,0.000000,6,182.0950,0,1998,1,2,0,732.3800
NCD19,8.93,1,0.000000,9,53.8614,1,1987,0,2,1,994.7052


In [265]:
df2['Outlet_Size']=le.fit_transform(df2['Outlet_Size'])
df2['Outlet_Location_Type']=le.fit_transform(df2['Outlet_Location_Type'])
df2['Outlet_Type']=le.fit_transform(df2['Outlet_Type'])
df2['Item_Type']=le.fit_transform(df2['Item_Type'])
df2['Item_Fat_Content']=df2['Item_Fat_Content'].map({'Low Fat':1,'Regular':2,'LF':1,'reg':2,'low fat':1})
df2['Outlet_Identifier']=le.fit_transform(df2['Outlet_Identifier'])

In [266]:
Y=df1.Item_Outlet_Sales
features=['Item_Weight','Item_Fat_Content','Item_Visibility','Item_Type','Item_MRP','Outlet_Identifier','Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type','Outlet_Type']

In [267]:
X=df1[features].copy()
X_test=df2[features].copy()
X_test['Outlet_Identifier']

Item_Identifier
FDW58    9
FDW14    2
NCN55    0
FDQ58    2
FDY38    5
        ..
FDB58    8
FDD47    3
NCO17    7
FDJ26    2
FDU37    7
Name: Outlet_Identifier, Length: 5681, dtype: int64

In [268]:
X_test.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
Item_Identifier,,,,,,,,,,
FDW58,20.750,1,0.007565,13,107.8622,9,1999,1,0,1
FDW14,8.300,2,0.038428,4,87.3198,2,2007,1,1,1
NCN55,14.600,1,0.099575,11,241.7538,0,1998,1,2,0
FDQ58,7.315,1,0.015388,13,155.0340,2,2007,1,1,1
FDY38,10.500,2,0.118599,4,234.2300,5,1985,1,2,3


In [269]:
from sklearn.model_selection import train_test_split
X_train,X_valid,Y_train,Y_valid=train_test_split(X,Y,train_size=0.8,test_size=0.2,random_state=0)

In [270]:
from sklearn.ensemble import RandomForestRegressor

# Define the models
model_1 = RandomForestRegressor(n_estimators=50, random_state=0)
model_2 = RandomForestRegressor(n_estimators=100, random_state=0)
model_3 = RandomForestRegressor(n_estimators=100, criterion='mae', random_state=0)
model_4 = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=0)
model_5 = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=0)

models = [model_1, model_2, model_3, model_4, model_5]

In [271]:
from sklearn.metrics import mean_absolute_error

# Function for comparing different models
def score_model(model, X_t=X_train, X_v=X_valid, y_t=Y_train, y_v=Y_valid):
    model.fit(X_t, y_t)
    preds = model.predict(X_v)
    return mean_absolute_error(y_v, preds)

for i in range(0, len(models)):
    mae = score_model(models[i])
    print("Model %d MAE: %d" % (i+1, mae))

Model 1 MAE: 815
Model 2 MAE: 812
Model 3 MAE: 812
Model 4 MAE: 789
Model 5 MAE: 774


In [272]:
my_model = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=0)
my_model.fit(X,Y)

# Generate test predictions
preds_test = my_model.predict(X_test)

# Save predictions in format used for competition scoring
output = pd.DataFrame({'Item_Identifier': X_test.index,
                       'Outlet_Identifier': le.inverse_transform(X_test['Outlet_Identifier']),
                       'Item_Outlet_Sales': preds_test})
output.to_csv('submission5.csv', index=False)